Download instructions and competition/data description could be found at https://competitions.codalab.org/competitions/17094#learn_the_details

Use `iw.path.list_raw_path` to save raw LiTS data.

In [ ]:
import os
from os.path import join as jp

import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from skimage.measure import label

from dpipe.io import load
from dpipe.medim.visualize import slice3d, gray_image_bright_colored_mask
from dpipe.medim.box import mask2bounding_box, add_margin, limit_box, box2slices
from iw.io import save_nii

In [ ]:
# from iw.path_local import lits_raw_path, lits_data_path
from iw.path import lits_raw_path, lits_data_path

data_inp = '{}media/nas/01_Datasets/CT/LITS/Training Batch {}/'
data_out_tumor = lits_data_path

In [ ]:
all_fnames = list(os.listdir(data_inp.format(lits_raw_path, 1)) + os.listdir(data_inp.format(lits_raw_path, 2)))
segm_names = []
for name in all_fnames:
    if 'segm' in name:
        segm_names.append(name)

In [ ]:
tumor_records = []

for i in tqdm_notebook(range(len(segm_names))):
    segm_name = f'segmentation-{i}.nii'
    img_name = f'volume-{i}.nii'
    
    n_batch = 1 if i <= 27 else 2
    
    segm = load(jp(data_inp.format(lits_raw_path, n_batch), segm_name))
    img = load(jp(data_inp.format(lits_raw_path, n_batch), img_name))

    segm_liver = segm == 1    
    box = limit_box(add_margin(mask2bounding_box(segm_liver), margin=5), limit=segm_liver.shape)
    
    tumor_mask = segm == 2
    tumor_mask_cropped = tumor_mask[box2slices(box)]
    img_cropped = img[box2slices(box)]
    
    save_nii(jp(data_out_tumor, segm_name), np.float32(tumor_mask_cropped))
    save_nii(jp(data_out_tumor, img_name), np.float32(img_cropped))
    _, n_tum = label(tumor_mask_cropped, return_num=True)
    tumor_records.append({'id': i, 'n_batch': n_batch, 'CT': img_name, 'target': segm_name, 'n_tumors': n_tum})

metadata_tumor = pd.DataFrame.from_records(tumor_records, index='id')
metadata_tumor.to_csv(path_or_buf=jp(data_out_tumor, 'metadata.csv'), index_label='id')